## Trying pmc_id_converter

> "pmc_id_converter for ID converter between PMID, PMCID and DOI"

Here the demonstration of this package is done in temporary sessions served via the MyBinder service, and so if you make anything useful save it back to your local machine ASAP.

The package is already installed in sessions that come up if you launched from [here](https://github.com/fomightez/pmc_id_converter_demo-binder). Otherwise, you'd want to run `%pip install pmc-id-converter` in your Jupyter Notebook file.

--------

# Use `pmc_id_converter` via Command Line interface

NOTE: 
In these demonstrations the exclamation point is used to send commands to the command line. You would not not need this in the terminal/console. **In other words, in most cases you'll want to actually delete the exclamation point shown at the start of these commands to use on the REAL command line.**

Running `pmc_idconv --help` shows you several command examples and options for usage

In [1]:
!pmc_idconv --help

Usage: pmc_idconv [OPTIONS] [IDS]...

  ID converter between PMID, PMCID and DOI

Options:
  --version           Show the version and exit.
  -o, --outfile TEXT  the output filename [stdout]
  -h, -?, --help      Show this message and exit.

  

  Examples:
      pmc_idconv --help
      pmc_idconv 30003000                     [PMID]
      pmc_idconv PMC6039336                   [PMCID]
      pmc_idconv 10.1007/s13205-018-1330-z    [DOI]
      pmc_idconv 30003000 30003001 30003002   [BATCH]
      pmc_idconv 30003000 30003001 -o out.jl  [FILE]  

  Contact: suqingdong <suqingdong1114@gmail.com> 


Let's step through running those examples handling identifier conversion here as a way to cover the use of this on the command line.

In [2]:
!pmc_idconv 30003000

{"doi": "10.1007/s13205-018-1330-z", "pmcid": "PMC6039336", "pmid": 30003000, "requested-id": "30003000"}


In [3]:
!pmc_idconv PMC6039336

{"doi": "10.1007/s13205-018-1330-z", "pmcid": "PMC6039336", "pmid": 30003000, "requested-id": "PMC6039336"}


In [4]:
!pmc_idconv 10.1007/s13205-018-1330-z 

{"doi": "10.1007/s13205-018-1330-z", "pmcid": "PMC6039336", "pmid": 30003000, "requested-id": "10.1007/s13205-018-1330-z"}


The fourth example involving identifiers demonstrates BATCH conversion of multiple identifiers.

In [5]:
!pmc_idconv 30003000 30003001 30003002

{"doi": "10.1007/s13205-018-1330-z", "pmcid": "PMC6039336", "pmid": 30003000, "requested-id": "30003000"}
{"doi": "10.1002/open.201800095", "pmcid": "PMC6031859", "pmid": 30003001, "requested-id": "30003001"}
{"doi": "10.1002/open.201800044", "pmcid": "PMC6031856", "pmid": 30003002, "requested-id": "30003002"}


The fifth example from the usage involving identifiers is meant to demonstrate sending the the results to a file. The related example on the repo [there]((https://github.com/suqingdong/pmc_id_converter#command-line) looks more fully realized version of this and so I suggest using that instead:

```shell
# Output to a file
pmc_idconv 30003000 30003001 30003002 -o out.json
```

In [6]:
!pmc_idconv 30003000 30003001 30003002 -o out.json

[2025-10-22 15:20:29 Main cli DEBUG MainThread:45] save file to: out.json


At this time I cannot explain why it shows `DEBUG MainThread:45` because everything seems to work so I suggest ignoring that for now.  
Run the next command to demonstratie the last example worked:

In [7]:
!head out.json

{"doi": "10.1007/s13205-018-1330-z", "pmcid": "PMC6039336", "pmid": 30003000, "requested-id": "30003000"}
{"doi": "10.1002/open.201800095", "pmcid": "PMC6031859", "pmid": 30003001, "requested-id": "30003001"}
{"doi": "10.1002/open.201800044", "pmcid": "PMC6031856", "pmid": 30003002, "requested-id": "30003002"}


That shows the content of the file generated file `out.json` are the expected results given the output from `pmc_idconv 30003000 30003001 30003002` earlier.

----------------

# Use `pmc_id_converter` via Python

In addition to the command line, the Usage notes featured in [suqingdong's GitHub repo for pmc_id_converter](https://github.com/suqingdong/pmc_id_converter#id-converter-between-pmid-pmcid-and-doi) show you can use it via Python as well.

I think this offers much more functionality than the command line if you are going to be using this with Python or Jupyter with a Python-based kernel. However, the offered demonstrations are lacking for novices not looking to dig through the code. I will expand greatly on these and give examples of how you can use this utility in your Python ecosystem. In particular, I'll include use with Pandas that is common package in data science work within the Python ecosystem.

??????

------

Enjoy!